In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval
from tqdm import tqdm
tqdm.pandas()
import re

#### part 1: getting NER markup on the clean text

In [3]:
path = '/home/sergey/Python_projects/RU_NER/Project_Data/Data/CommonVoice/Converted_Common_Voice_ru.csv'

In [4]:
df = pd.read_csv(path, index_col=0, low_memory=False)

In [6]:
df.head()

,id,sentence,transcription,difference,label,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,18849003,Владимир вытащил пробку.,владимир вытащил пробку,[],validated,2,0,twenties,male,NaN,NaN,ru,NaN
1,18849004,Слово имеет уважаемый представитель Республики...,слово имеет уважаемый представитель в республи...,"['в', 'корее', 'корея', 'республике', 'республ...",validated,2,0,twenties,male,NaN,NaN,ru,NaN
2,18849005,Совет Безопасности приступает к рассмотрению п...,совет безопасности приступает к рассмотрению п...,[],validated,2,0,twenties,male,NaN,NaN,ru,NaN
3,18849006,Не лезь!,не лезь,[],validated,2,0,twenties,male,NaN,NaN,ru,NaN
4,18849007,И даже смеяться перестали.,даже смеяться перестали,['и'],validated,2,0,twenties,male,NaN,NaN,ru,NaN


In [5]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [6]:
test_sentence = df['sentence'][1]

In [7]:
ner_results = nlp(test_sentence)
print(ner_results)

[{'entity_group': 'LOC', 'score': 0.99750185, 'word': 'Республики Корея', 'start': 36, 'end': 52}]


In [ ]:
df['NER'] = df['sentence'].progress_apply(lambda x: nlp(x))

In [ ]:
#df.to_csv('/home/sergey/Python_projects/RU_NER/Data/CommonVoice/NER_Common_Voice_ru.csv')

##### part2: adjusting NER markup so it is accurate for transcribed texts

In [147]:
df = pd.read_csv('NER_Common_Voice_ru.csv', index_col=0, low_memory=False)

Сначала вычищу датасет, уберу дубликаты (разноголосые записи), пропуски распознавания (30 записей было сделано шепотом, и их не распознало):

In [148]:
dups_mask = df.duplicated(subset='sentence')
df[dups_mask].sort_values(by='sentence').head(10)

,id,sentence,transcription,difference,label,up_votes,down_votes,age,gender,accents,variant,locale,segment,NER
85702,24967389,- Сейчас заканчиваю.,сейчас заканчиваю,[''],validated,2,0,twenties,male,NaN,NaN,ru,NaN,"[{'entity_group': 'MISC', 'score': 0.7337075, ..."
92469,25969972,- Сейчас заканчиваю.,сейчас заканчиваю,[''],validated,2,0,thirties,male,NaN,NaN,ru,NaN,"[{'entity_group': 'MISC', 'score': 0.7337075, ..."
56669,20801207,"- Смотрите же, братцы, не отставать!",смотрите же братцы не отставать,[''],dev,2,0,twenties,male,NaN,NaN,ru,NaN,[]
91101,25835936,"- Смотрите же, братцы, не отставать!",смотрите же братцы не отставать,[''],validated,2,0,twenties,male,NaN,NaN,ru,NaN,[]
8920,18912323,Cтесненное положение перестало в последнее вре...,стеснённое положение перестало в последнее вре...,"['cтесненное', 'стеснённое']",validated,2,1,twenties,female,NaN,NaN,ru,NaN,"[{'entity_group': 'MISC', 'score': 0.6892858, ..."
20679,18934178,Cтесненное положение перестало в последнее вре...,стеснённое положение перестало в последнее вре...,"['cтесненное', 'стеснённое']",validated,2,0,fifties,female,NaN,NaN,ru,NaN,"[{'entity_group': 'MISC', 'score': 0.6892858, ..."
24075,18948049,Cтесненное положение перестало в последнее вре...,стеснённое положение перестало в последнее вре...,"['cтесненное', 'стеснённое']",validated,2,0,twenties,male,NaN,NaN,ru,NaN,"[{'entity_group': 'MISC', 'score': 0.6892858, ..."
15014,18922914,Cтесненное положение перестало в последнее вре...,стеснённое положение перестало в последнее вре...,"['cтесненное', 'стеснённое']",validated,2,0,twenties,female,NaN,NaN,ru,NaN,"[{'entity_group': 'MISC', 'score': 0.6892858, ..."
30414,18989967,Cтесненное положение перестало в последнее вре...,стеснённое положение перестало в последнее вре...,"['cтесненное', 'стеснённое']",train,2,0,twenties,male,NaN,NaN,ru,NaN,"[{'entity_group': 'MISC', 'score': 0.6892858, ..."
28962,18971196,Cтолько злобного презрения накопилось в душе м...,сколько злобного презрения накопилось в душе м...,"['cтолько', 'сколько']",validated,2,1,fourties,male,NaN,NaN,ru,NaN,[]


In [149]:
df.drop_duplicates(subset='sentence', ignore_index=True, inplace=True)

In [150]:
# проверяем что пропусков нет в изначлаьных данных:
df['sentence'].isna().value_counts()

False    46630
Name: sentence, dtype: int64

In [151]:
# пропуски в распознавании:
df['transcription'].isna().value_counts()

False    46600
True        30
Name: transcription, dtype: int64

In [152]:
cols = ['sentence', 'transcription']
df[df[cols].isna().any(axis=1)].head()

,id,sentence,transcription,difference,label,up_votes,down_votes,age,gender,accents,variant,locale,segment,NER
15617,19504492,"И она не виновата, и Ольга как будто бы тоже.",NaN,"['', 'будто', 'бы', 'виновата', 'и', 'как', 'н...",validated,2,1,NaN,NaN,NaN,NaN,ru,NaN,"[{'entity_group': 'PER', 'score': 0.9578805, '..."
33083,25919310,Это сильно затрудняет злоумышленнику обнаружен...,NaN,"['', 'данных', 'затрудняет', 'злоумышленнику',...",validated,2,0,twenties,male,NaN,NaN,ru,NaN,[]
33480,26316660,Он потребовал нас к коменданту.,NaN,"['', 'к', 'коменданту', 'нас', 'он', 'потребов...",validated,2,0,NaN,NaN,NaN,NaN,ru,NaN,[]
33515,26353492,Марья Ивановна заплакала.,NaN,"['', 'заплакала', 'ивановна', 'марья']",dev,2,0,twenties,male,NaN,NaN,ru,NaN,"[{'entity_group': 'PER', 'score': 0.99782026, ..."
34076,27207801,Гей!,NaN,"['', 'гей']",validated,2,0,twenties,male,NaN,NaN,ru,NaN,"[{'entity_group': 'MISC', 'score': 0.68143344,..."


In [153]:
df.dropna(subset=cols, inplace=True)
df.reset_index(drop=True, inplace=True)

In [154]:
markup = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

In [155]:
df['NER'] = df['NER'].apply(literal_eval)

In [156]:
# Concatenate strange tokenization (В + ##енесуэла = Венесуэла), and change the NER results accordingly,
# changing start/end according to new tokenization, as well as using mean score for combined entity
def ner_concat(sentence, ner):
    del_index = []
    merged_ner = []
    
    for index in range(len(ner)):
        if '#' not in ner[index]['word']:
            merged_ner.append(ner[index])
        else:
            ner[index]['word'] = ner[index]['word'].strip('#')
            if merged_ner and ner[index]['start'] == merged_ner[-1]['end']:
                merged_ner[-1]['word'] += ner[index]['word']
                merged_ner[-1]['score'] = np.mean([merged_ner[-1]['score'], ner[index]['score']])
                merged_ner[-1]['end'] = ner[index]['end']
            else:
                merged_ner.append(ner[index])
                
    tokens = re.sub(r'[^\w\s]', '', sentence).split()
    
    for item in merged_ner:
        remove_flag = False
        for word in re.sub(r'[^\w\s]', '', item['word']).split():
            if word not in tokens:
                remove_flag = True
        if remove_flag:
            merged_ner.remove(item)

    return merged_ner

In [160]:
df['NER'] = df.apply(lambda row: ner_concat(row['sentence'], row['NER']), axis=1)

In [161]:
score_treshold = 0.85
misc_treshold = 0.95

# Remove bizzare results, which mostly have low score (threshold)
# using higher threshold for defined groups (LOC/PERS/ORG)
def ner_cleanup(ner):
    del_index = []
    for index in range(len(ner)):
        if re.sub(r'[^\w\s]', '', ner[index]['word']) == '' : del_index.append(index)
        elif ner[index]['score'] < score_treshold : del_index.append(index)
        elif (ner[index]['entity_group'] == 'MISC' and ner[index]['score'] < misc_treshold):
            del_index.append(index)
    for ele in sorted(del_index, reverse=True):
        del ner[ele]
        
    return ner

In [162]:
df['NER'] = df['NER'].apply(lambda x: ner_cleanup(x))

In [163]:
# Get a markup similar to other datasets, produces a list according to 'markup' for each sentence

def convert_markup(sentence, ner):
    tokens = re.sub(r'[^\w\s]', '', sentence).split()
    markup_list = [0] * len(tokens)
    
    for dic in ner:
        entity_start = dic['start']
        entity_end = dic['end']
        entity_type = dic['entity_group']
        entity_tokens = re.sub(r'[^\w\s]', '', sentence[entity_start:entity_end]).split()
        entity_length = len(entity_tokens)
        if entity_length == 1:
            markup_index = re.sub(r'[^\w\s]', '', sentence).split().index(entity_tokens[0])
            markup_list[markup_index] = markup['B-' + entity_type]
            
        else:
            markup_index = [re.sub(r'[^\w\s]', '', sentence).split().index(token) for token in entity_tokens]
            markup_list[markup_index[0]] = markup['B-' + entity_type]
            markup_list[markup_index[1]:markup_index[0]+entity_length] = [markup['I-' + entity_type] for _ in range(entity_length-1)]
            
    return markup_list

In [164]:
df['NER_markup'] = df.apply(lambda row: convert_markup(row['sentence'], row['NER']), axis=1)

In [165]:
df.drop(['up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant', 'locale', 'segment'], axis=1, inplace=True)

In [166]:
df.head()

,id,sentence,transcription,difference,label,NER,NER_markup
0,18849003,Владимир вытащил пробку.,владимир вытащил пробку,[],validated,"[{'entity_group': 'PER', 'score': 0.9800292, '...","[1, 0, 0]"
1,18849004,Слово имеет уважаемый представитель Республики...,слово имеет уважаемый представитель в республи...,"['в', 'корее', 'корея', 'республике', 'республ...",validated,"[{'entity_group': 'LOC', 'score': 0.99750185, ...","[0, 0, 0, 0, 5, 6]"
2,18849005,Совет Безопасности приступает к рассмотрению п...,совет безопасности приступает к рассмотрению п...,[],validated,"[{'entity_group': 'ORG', 'score': 0.9949068, '...","[3, 4, 0, 0, 0, 0, 0, 0, 0]"
3,18849006,Не лезь!,не лезь,[],validated,[],"[0, 0]"
4,18849007,И даже смеяться перестали.,даже смеяться перестали,['и'],validated,[],"[0, 0, 0, 0]"


Финальная проверка разметки, длинна распознанных предложений и разметки должна совпадать с чистым предложением:

In [167]:
def length_check(col1, col2):
    return 'ok' if len(col1.split()) == len(col2.split()) else 'WRONG'

In [168]:
df['len'] = df.apply(lambda row: length_check(row['sentence'], row['transcription']), axis=1)

In [169]:
df.len.value_counts()

ok       36208
WRONG    10392
Name: len, dtype: int64

In [171]:
df = df[df['len'] == 'ok']
df.reset_index(drop=True, inplace=True)

In [172]:
def length_check(col1, ner):
    return 'ok' if len(col1.split()) == len(ner) else 'WRONG'

In [173]:
df['len'] = df.apply(lambda row: length_check(row['transcription'], row['NER_markup']), axis=1)

/tmp/ipykernel_57046/3562682296.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['len'] = df.apply(lambda row: length_check(row['transcription'], row['NER_markup']), axis=1)


In [174]:
df.len.value_counts()

ok       36061
WRONG      147
Name: len, dtype: int64

In [175]:
df = df[df['len'] == 'ok']
df.reset_index(drop=True, inplace=True)

In [176]:
def print_tags_frequency(tags):
    freq = {}
    for row in tags: 
        for tag in set(row):
            if tag in freq:
                freq[tag] += 1
            else:
                freq[tag] = 1
    freq
    for item in sorted(freq.keys()):
        print(f'% of rows token {item} appears in is {freq[item]/len(tags)*100:.4f}')

In [177]:
print_tags_frequency(df.NER_markup)

% of rows token 0 appears in is 99.6312
% of rows token 1 appears in is 7.7452
% of rows token 2 appears in is 2.0743
% of rows token 3 appears in is 5.8623
% of rows token 4 appears in is 4.0126
% of rows token 5 appears in is 11.0147
% of rows token 6 appears in is 2.2850
% of rows token 7 appears in is 1.0815
% of rows token 8 appears in is 0.3799


In [179]:
df.to_csv('Edited_NER_anotation_Common_Voice.csv', index=False)